In [ ]:
# !pip3 install holidays

In [1]:
import pandas as pd
import holidays
import datetime
import calendar
import numpy as np

In [ ]:
# df = pd.read_csv('/home/faye/data/vehicles_2018_cleaned_dataset')

In [ ]:
# Converting date columns to datetime
# datetime_columns = ['DAYOFSERVICE', 'LASTUPDATE']
# for column in datetime_columns:
#     df[column] = pd.to_datetime(df[column])

In [ ]:
# Getting the Irish holidays - including bank holidays
# https://towardsdatascience.com/5-minute-guide-to-detecting-holidays-in-python-c270f8479387
# irish_holidays_2018 = []
# for date in holidays.Ireland(years=2018).items():
#     irish_holidays_2018.append(str(date[0]))

In [ ]:
# irish_holidays_2018

In [ ]:
# df['Is_HOLIDAY'] = [1 if str(val).split()[0] in irish_holidays_2018 else 0 for val in df['DAYOFSERVICE']]
# df['IS_WEEKDAY'] = [1 if int(val.weekday()) < 5 else 0 for val in df['DAYOFSERVICE']]
# df['DAYOFWEEK'] = [calendar.day_name[val.weekday()] for val in df['DAYOFSERVICE']]
# df['MONTHOFSERVICE'] = df['DAYOFSERVICE'].dt.strftime('%B')

### Calculating how long it took to get to the next stop
As we know, each tripid represents one trip at a certain point of the day. So what we need to do is take away the second stop from the first stop. **actualtime_arr from the second stop - actualtime_dep from the first stop**

**Prequisites:**
* Have each row by specific trip and date.

**Method:**
* Loop through every date and tripid:
    * Initiate an empty list called time_between
    * If tripid and date match in a row:
        * Store PROGRNUMBER in a list then 
        * Loop through the list with PROGRNUMBERS:
            * add to time between: the current progrnumber actualtime_arr MINUS progrnumber previous actualtime_dep
    * Add that populated list to the existing rows in loop
    * Replace rows according to the index
    <br>
    (May cause memory error but whatever we'll try this method)

In [2]:
leavetimes = pd.read_feather('/home/faye/data/leavetimes_cleaned_2.feather')

In [4]:
leavetimes['DAYOFSERVICE'] = pd.to_datetime(leavetimes['DAYOFSERVICE'])
leavetimes['DAYOFSERVICE'] = (leavetimes['DAYOFSERVICE'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

In [5]:
leavetimes = leavetimes.sort_values(by='PROGRNUMBER')

In [6]:
tripid = [list(leavetimes['TRIPID'].unique()), list(leavetimes['DAYOFSERVICE'].unique())]

In [7]:
# making new column with empty values
leavetimes['TOTALTIME'] = np.nan

In [8]:
# Going to try and define a function that would return time_between as a list instead
def list_time_between(prog_num):
    return [0 if stop == 1
           else
           (int(row['ACTUALTIME_ARR'].loc[row['PROGRNUMBER']==stop])) - 
            (int(row['ACTUALTIME_DEP'].loc[row['PROGRNUMBER']==(prog_num[prog_num.index(stop)-1])]))
           for stop in prog_num]


In [18]:
for id_, date in zip(tripid[0], tripid[1]):
    row = leavetimes.loc[(leavetimes['TRIPID']==id_) & (leavetimes['DAYOFSERVICE']==date)]
    time_between = list_time_between(list(row['PROGRNUMBER']))
    row.drop('TOTALTIME', axis=1, inplace=True)
    row['TOTALTIME'] = time_between
    leavetimes.loc[row.index, 'TOTALTIME'] = row['TOTALTIME']
    print(leavetimes.loc[row.index, 'TOTALTIME'])
#     leavetimes.combine_first(row)

/home/faye/.local/lib/python3.6/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/faye/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


11423513      0.0
11423514    113.0
11423515     17.0
11423516     27.0
11423517      8.0
            ...  
11424730     67.0
11424731     19.0
11424732     49.0
11424733     31.0
11424734     22.0
Name: TOTALTIME, Length: 84, dtype: float64


/home/faye/.local/lib/python3.6/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/faye/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


28417249      0.0
22329151     36.0
22381681     82.0
22381685     24.0
22381688    108.0
22381692     50.0
22381695    125.0
22601755     97.0
22381701     56.0
22381705     34.0
22381708     54.0
22381712     54.0
28420196     40.0
22381717     56.0
22382802     39.0
22382805     93.0
22602880     33.0
28400449     42.0
22382815     89.0
22382819     38.0
22382822     69.0
22382826     62.0
22382828     59.0
22602881     41.0
22382835     23.0
22382839     22.0
22382842     63.0
22382846     97.0
22382849    147.0
22382852    264.0
22382856    181.0
22382859     84.0
22382863    143.0
Name: TOTALTIME, dtype: float64


/home/faye/.local/lib/python3.6/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/faye/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


...

Series([], Name: TOTALTIME, dtype: float64)


/home/faye/.local/lib/python3.6/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Series([], Name: TOTALTIME, dtype: float64)


/home/faye/.local/lib/python3.6/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


KeyboardInterrupt: 

In [15]:
leavetimes.isnull().sum()

DAYOFSERVICE               0
TRIPID                     0
PROGRNUMBER                0
STOPPOINTID                0
VEHICLEID                  0
PLANNEDTIME_ARR            0
ACTUALTIME_ARR             0
PLANNEDTIME_DEP            0
ACTUALTIME_DEP             0
TOTALTIME          116948672
dtype: int64